In [1]:
#import libraries
import ee
import os
import geemap
import datetime
import plotly.graph_objects as go
from ipyleaflet import WidgetControl
import ipywidgets as widgets
from ipywidgets import HTML


In [2]:
#start session in GEE 
ee.Authenticate()

project = 'ee-vl99956018'
ee.Initialize(project=project)


In [3]:
#before fire date range
pre_start_date = ee.Date('2025-01-01')
pre_end_date = pre_start_date.advance(4, 'day')  # 2025-01-05

#after fire date range
pos_start_date = pre_end_date.advance(1, 'day')  # 2025-01-06
pos_end_date = pos_start_date.advance(10, 'day')  # 2025-01-16

#coordinates of interest
latitude = 34.092615
longitude = -118.532875

#Region of interest (ROI) with a buffer of 15km
roi = ee.Geometry.Point(longitude, latitude).buffer(15000)


In [4]:
# Define palette color for visualization (RGB, NDVI and NBR)
# and color bar settings

rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3500,  
    'gamma': 1.2
}


ndvi_vis = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'green']
}

nbr_vis = {
    'min': -1,
    'max': 1,
    'palette': ['white', 'black', 'red']
    
}

diff_nbr_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['blue', 'white', 'red']
}

#Initialize map with center and zoom
Map = geemap.Map(center=[latitude, longitude], zoom=13)


In [5]:
#define functions

def get_all_collections(start_date: str, end_date: str) -> dict:
    """
    Functions to get Sentinel-2 image collection, NDVI and NBR for a given date range.
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
    Returns:
        dict: Dictionary containing the image collection, NDVI, and NBR images.
    """

    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, end_date) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

    # Select relevant bands and compute median
    image = collection.select('B8', 'B12', 'B4', 'B3', 'B2').median().clip(roi)

    # NDVI
    ndvi = collection.select(['B8', 'B4']) \
        .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI')) \
        .median().clip(roi)

    # NBR
    nbr = collection.select(['B8', 'B12']) \
        .map(lambda img: img.normalizedDifference(['B8', 'B12']).rename('NBR')) \
        .median().clip(roi)

    return {
        'image': image,
        'ndvi': ndvi,
        'nbr': nbr
    }


# Export function to Google Drive
# def export_to_drive(image: ee.Image, image_title: str, folder_name='GEE_exports'):
#     """Exports an image to Google Drive.
#     Args:
#         image (ee.Image): The image to export.
#         image_title (str): The title for the exported image.
#         folder_name (str): The name of the folder in Google Drive where the image will be saved.
#     """
#     task = ee.batch.Export.image.toDrive(
#         image = image,
#         description = image_title,
#         folder = folder_name,
#         fileNamePrefix = image_title.lower().replace(' ', '_'), #standardize file name
#         scale = 10,  # Sentinel-2 resolution
#         region = roi,
#         fileFormat = 'GeoTIFF',
#         maxPixels = 1e10
#     )
#     task.start()
#     print(f'Started export task for {image_title}. Check your Google Drive folder "{folder_name}" when complete.')


def export_to_drive_and_local(image: ee.Image, image_title: str, folder_name='GEE_exports'):
    """Exports an image to Google Drive and downloads it locally as GeoTIFF."""

    # ---------- DOWNLOAD LOCALLY FROM EARTH ENGINE ----------
    print(f"[→] Downloading {image_title} locally...")

    try:
        # Generate download URL directly from the image object
        url = image.getDownloadURL({
            'scale': 35,
            'region': roi,
            'format': 'GEO_TIFF'
        })

        # Download file
        import requests
        out_name = f"{image_title.lower().replace(' ', '_')}.tif"

        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(out_name, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"[✓] Download completed: {out_name}")

    except Exception as e:
        print(f"[X] Error downloading image locally: {e}")

# Create HTML labels for map output
def create_labels_html(legend_before: str, legend_after: str):

    antes_html = HTML(f'''<div style="position: absolute; top: 10px; left: 10px; z-index: 1000; 
                        background-color: white; padding: 5px; border-radius: 5px;
                        font-weight: bold; color: black;">{legend_before}</div>''')
                        
    depois_html = HTML(f'''<div style="position: absolute; top: 70px; right: 10px; z-index: 1000; 
                         background-color: white; padding: 5px; border-radius: 5px;
                         font-weight: bold; color: black;">{legend_after}</div>''')

    return antes_html, depois_html


# Get the most recent date of available images
def get_most_recent_date(start_date: str) -> str:
    """
    Retrieves the most recent date of Sentinel-2 images in the specified date range.
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format.
    Returns:
        str: Most recent date in 'YYYY-MM-DD' format.    
    """
    # Today's date
    today = datetime.date.today().isoformat()

    # Create image collection
    collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
        .filterDate(start_date, today) \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
        .sort('system:time_start', False)  

    # Pick the most recent image
    most_recent_image = collection.first()

    # Get the date of the most recent image
    most_recent_date = ee.Date(most_recent_image.get('system:time_start')).format('YYYY-MM-dd')
    return most_recent_date.getInfo()



In [6]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN RGB

map_rgb = Map

# Pre and post fire images in RGB
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['image']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']


# Add split map to compare before and after fire images in RGB
left_layer = geemap.ee_tile_layer(pre_image, rgb_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')

map_rgb.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_rgb.addLayer(pre_image, rgb_vis, 'Before Fire', False)
map_rgb.addLayer(pos_image, rgb_vis, 'After Fire', False)




# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_rgb.add_control(WidgetControl(widget=before_html, position='topleft'))
map_rgb.add_control(WidgetControl(widget=after_html, position='topright'))

map_rgb

# Export images to Google Drive (uncomment to use)
# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")




Map(center=[34.092615, -118.532875], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [11]:
import matplotlib.pyplot as plt
from osgeo import gdal
import requests
import ee

def download_image_as_pdf(image: ee.Image, image_title: str, scale=35):
    print(f"[→] Baixando {image_title} como TIFF...")

    try:
        url = image.getDownloadURL({
            'scale': scale,
            'region': roi,
            'format': 'GEO_TIFF'
        })

        tif_name = f"{image_title.lower().replace(' ', '_')}.tif"
        pdf_name = f"{image_title.lower().replace(' ', '_')}.pdf"

        # --- Download TIFF ---
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(tif_name, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"[✓] TIFF baixado: {tif_name}")

        # --- Load TIFF with GDAL ---
        ds = gdal.Open(tif_name)
        img = ds.ReadAsArray()

        # Rearrange bands from (bands, y, x) → (y, x, bands)
        img = img.transpose((1, 2, 0))

        # Select only RGB: last 3 bands (B4/B3/B2)
        rgb = img[:, :, -3:]

        # Normalize 0–255 (Matplotlib expects this)
        rgb_norm = (255 * (rgb - rgb.min()) / (rgb.max() - rgb.min())).astype('uint8')

        # --- Save as PDF ---
        print("[→] Convertendo para PDF...")
        plt.figure(figsize=(10, 10))
        plt.imshow(rgb_norm)
        plt.axis("off")
        plt.savefig(pdf_name, format="pdf", bbox_inches="tight")
        plt.close()

        print(f"[✓] PDF gerado: {pdf_name}")

        return pdf_name

    except Exception as e:
        print(f"[X] Erro: {e}")


download_image_as_pdf(pre_image, "before_fire_RGB_image")


[→] Baixando before_fire_RGB_image como TIFF...
[✓] TIFF baixado: before_fire_rgb_image.tif
[→] Convertendo para PDF...


Warning 1: before_fire_rgb_image.tif: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


[✓] PDF gerado: before_fire_rgb_image.pdf


'before_fire_rgb_image.pdf'

In [18]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NDVI

map_ndvi = Map


# Pre and post fire images in NDVI
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['ndvi']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['ndvi']


# Add split map to compare before and after fire images in NDVI
left_layer = geemap.ee_tile_layer(pre_image, ndvi_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, ndvi_vis, 'After Fire')

map_ndvi.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_ndvi.addLayer(pre_image, ndvi_vis, 'Before Fire', False)
map_ndvi.addLayer(pos_image, ndvi_vis, 'After Fire', False)


# NDVI color bar
map_ndvi.add_colorbar(
    vis_params=ndvi_vis,
    label='NDVI',
    layer_name='NDVI',
    orientation='vertical',  
    position='bottomright'
    )

# Add HTML labels to the map
before_html, adter_html = create_labels_html('Before', 'After')
map_ndvi.add_control(WidgetControl(widget=before_html, position='topleft'))
map_ndvi.add_control(WidgetControl(widget=adter_html, position='topright'))

map_ndvi

# Export images to Google Drive (uncomment to use)
# export_to_drive(pre_image, "before_fire_NDVI_image")
# export_to_drive(pos_image, "after_fire_NDVI_image")


Map(bottom=418983.0, center=[34.05408166391527, -118.5029983520508], controls=(ZoomControl(options=['position'…

In [19]:
# COMPUTE IMAGES FOR BEFORE AND AFTER FIRE IN NBR

map_nbr = Map

# Pre and post fire images in NBR
pre_result = get_all_collections(pre_start_date, pre_end_date)
pre_image = pre_result['nbr']

pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['nbr']


# Calculate difference between pre and post fire images in NBR
dnbr = pre_image.subtract(pos_image)

# Define a threshold to identify burned areas
burned_mask = dnbr.gt(0.3)

# Calculate total burned area (in hec)
pixel_area = burned_mask.multiply(ee.Image.pixelArea()).divide(10000)  # m^2 to hec
area_burned = pixel_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=roi,
    scale=30,
    maxPixels=1e13
)

# Print burned area
print("Burned Area (hec):", round(area_burned.getInfo()['NBR'],2))


# Add split map to compare before and after fire images in NBR
left_layer = geemap.ee_tile_layer(pre_image, nbr_vis, 'Before Fire')
right_layer = geemap.ee_tile_layer(pos_image, nbr_vis, 'After Fire')


map_nbr.split_map(left_layer=left_layer, right_layer=right_layer)


# Add layers to the map (optional, can be toggled on/off in the layer control)
map_nbr.addLayer(pre_image, nbr_vis, 'Before Fire', False)
map_nbr.addLayer(pos_image, nbr_vis, 'After Fire', False)


# NBR color bar
map_nbr.add_colorbar(
    vis_params=nbr_vis,
    label='NBR',
    layer_name='NBR',
    orientation='vertical',
    position='bottomleft'
)

# Add HTML labels to the map
before_html, after_html = create_labels_html('Before', 'After')
map_nbr.add_control(WidgetControl(widget=before_html, position='topleft'))
map_nbr.add_control(WidgetControl(widget=after_html, position='topright'))



# Map for difference in NBR
Map_diff = geemap.Map(center=[latitude, longitude], zoom=13)
Map_diff.addLayer(dnbr, diff_nbr_vis, 'ΔNBR')

display(widgets.VBox([Map, Map_diff]))


# Export images to Google Drive (uncomment to use)
# export_to_drive(pre_image, "before_fire_NBR_image")
# export_to_drive(pos_image, "after_fire_NBR_image")
# export_to_drive(dnbr, "Difference NBR")




Burned Area (hec): 9357.08


In [ ]:
from scipy.signal import savgol_filter

def get_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def get_nbr(image):
    nbr = image.normalizedDifference(['B12', 'B8']).rename('NBR')
    return image.addBands(nbr)

def ndvi_nbr_plot(start_date, end_date):
    # Carrega coleção Sentinel-2
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterDate(start_date, end_date)
                    .filterBounds(roi)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
                    .map(get_ndvi)
                    .map(get_nbr)
                    .select(['NDVI', 'NBR']))

    # Função para extrair NDVI e NBR médios e data como Feature
    def reduce_img(img):
        stats = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        )
        return ee.Feature(None, {
            'date': img.date().format('YYYY-MM-dd'),
            'NDVI': stats.get('NDVI'),
            'NBR': stats.get('NBR')
        })


    features = collection.map(reduce_img)


    stats_list = features.aggregate_array('date').getInfo()
    ndvi_values = features.aggregate_array('NDVI').getInfo()
    nbr_values = features.aggregate_array('NBR').getInfo()

    # --- Smooth curves ---
    window = 21     # must be odd, increase to smooth more
    poly = 10       # polynomial order

    ndvi_smooth = savgol_filter(ndvi_values, window_length=window, polyorder=poly)
    nbr_smooth  = savgol_filter(nbr_values,  window_length=window, polyorder=poly)


    # Convert to datetime
    dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in stats_list]

    # Plot NDVI and NBR curves in the same figure
    fig = go.Figure()
    
    # Add NDVI trace
    fig.add_trace(go.Scatter(
        x=dates,
        y=ndvi_values,
        mode='lines+markers',
        name='NDVI',
        line=dict(color='green', width=2),
        marker=dict(size=8, color='green'),
        yaxis='y1'
    ))
    
    # Add NDVI trace (smoothed)
    fig.add_trace(go.Scatter(
        x=dates,
        y=ndvi_smooth,
        mode='lines',
        name='NDVI (smoothed)',
        line=dict(color='green', width=2)
    ))

    # Add NBR trace
    fig.add_trace(go.Scatter(
        x=dates,
        y=nbr_values,
        mode='lines+markers',
        name='NBR',
        line=dict(color='red', width=2),
        marker=dict(size=8, color='red'),
        yaxis='y1'
    ))

    # Add NBR trace (smoothed)
    fig.add_trace(go.Scatter(
        x=dates,
        y=nbr_smooth,
        mode='lines',
        name='NBR (smoothed)',
        line=dict(color='red', width=2)
    ))


    # Add vertical line at 2025-01-07
    fig.add_vline(
        x=datetime.datetime(2025, 1, 7).timestamp() * 1000,  # Convert to milliseconds since epoch
        line_width=2,
        line_dash="dash",
        line_color="blue",
        annotation_text="First Fire detected",
        annotation_position="top right"
    )

    fig.update_layout(
        title='NDVI and NBR TimeSeries',
        xaxis_title='Date',
        yaxis_title='Index Value',
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=True, title='Index Value'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        template='plotly_white',
        hovermode='x unified'
    )

    # Save plot
    os.makedirs("results", exist_ok=True)
    fig.write_html("results/ndvi_nbr_plot.html")
    print("Graphic saved at 'results/ndvi_nbr_plot.html'")
    fig.show(renderer = "browser")




Graphic saved at 'results/ndvi_nbr_plot.html'
Opening in existing browser session.


In [ ]:
curve_start_date = '2024-10-01'
curve_end_date = '2025-04-01'

# Run the function
ndvi_nbr_plot(curve_start_date, curve_end_date)

Graphic saved at 'results/ndvi_nbr_plot.html'
Opening in existing browser session.


In [20]:
pos_result = get_all_collections(pos_start_date, pos_end_date)
pos_image = pos_result['image']

recent_date = get_most_recent_date('2025-04-01')
today = datetime.date.today().isoformat()

today_result = get_all_collections(recent_date, today)
today_image = today_result['image']

print(f"The most recent date is {recent_date} for a cloud coverage of 10% or less.")

left_layer = geemap.ee_tile_layer(pos_image, rgb_vis, 'After Fire')
right_layer = geemap.ee_tile_layer(today_image, rgb_vis, 'Today')

Map.split_map(left_layer=left_layer, right_layer=right_layer)

# Adicionar controle de camadas para alternar entre visualizações
Map.addLayer(pos_image, rgb_vis, 'After Fire', False)
Map.addLayer(today_image, rgb_vis, 'Today', False)

# export_to_drive(pre_image, "before_fire_RGB_image")
# export_to_drive(pos_image, "after_fire_RGB_image")


# Adicionar controles ao mapa
antes_html, depois_html = create_labels_html('After', 'Today')
Map.add_control(WidgetControl(widget=antes_html, position='topleft'))
Map.add_control(WidgetControl(widget=depois_html, position='topright'))

Map

The most recent date is 2025-11-10 for a cloud coverage of 10% or less.


Map(bottom=1675032.0, center=[34.05408166391527, -118.5029983520508], controls=(ZoomControl(options=['position…